# **Lexical and Semantic Search with Elasticsearch**

In the following examples, we will explore various approaches to retrieving information using Elasticsearch - focusing specifically on full text search, semantic search, and a hybrid combination of both.

To accomplish this, this example demonstrates various search scenarios on a dataset generated to simulate e-commerce product information.

This dataset contains over 2,500 products, each with a description. These products are categorized into 76 distinct product categories, with each category containing a varying number of products. 

Here is a sample of an object from the dataset:

```json
 {
    "product": "Samsung 49-inch Curved Gaming Monitor",
    "description": "is a curved gaming monitor with a high refresh rate and AMD FreeSync technology.",
    "category": "Monitors"
}

```

We will consume the dataset from a JSON file into Elasticsearch using modern consumption patterns. We will then perform a series of search operations to demonstrate the different search strategies.



## **🧰 Requirements**

For this example, you will need:

- Python 3.11 or later
- The Elastic Python client
- Elastic 9.0 deployment or later on either a local, cloud, or serverless environment


We'll be using an [Elastic Serverless](https://www.elastic.co/docs/deploy-manage/deploy/elastic-cloud/serverless) instance. You can use a [free trial here](https://cloud.elastic.co/registration?onboarding_token=vectorsearch&utm_source=github&utm_content=elasticsearch-labs-notebook) to get started.

## Setup Elasticsearch environment:

To get started, we'll need to connect to our Elastic deployment using the Python client.

Because we're using an Elastic Cloud deployment, we'll use the **Elastic Cloud Endpoint** and **Elastic Cloud API Key** to identify our deployment. These may be found within Kibana by following the instructions [here](https://www.elastic.co/docs/deploy-manage/api-keys/elastic-cloud-api-keys).


In [ ]:
%pip install elasticsearch pandas IPython -q

### Import the required packages
We will import the following packages:
- `Elasticsearch`: a client library for Elasticsearch actions
- `bulk`: a function to perform Elasticsearch actions in bulk
- `getpass`: a module for receiving Elasticsearch credentials via text prompt
- `json`: a module for reading and writing JSON data
- `pandas`, `display`, `Markdown`: for data visualization and markdown formatting



In [ ]:
# import the Elasticsearch client and bulk function
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

# import getpass module to handle Auth input
import getpass

# import json module to read JSON file of products
import json  # module for handling JSON data

# display search results in a table
import pandas as pd
from IPython.display import display, Markdown

### 📚 Instantiating the Elasticsearch Client

First we prompt the user for their Elastic Endpoint URL and Elastic API Key.
Then we create a `client` object that instantiates an instance of the `Elasticsearch` class.
Lastly, we verify that our client is connected to our Elasticsearch instance by calling `client.ping()`.
> 🔐 *NOTE: `getpass` enables us to securely prompt the user for credentials without echoing them to the terminal, or storing it in memory.*

In [ ]:
# endpoint for Elasticsearch instance
ELASTIC_ENDPOINT = getpass.getpass("Enter Elastic Endpoint:  ")

# Elastic API key for Elasticsearch
ELASTIC_API_KEY = getpass.getpass("Enter Elastic API Key:  ")

# create the Elasticsearch client instance
client = Elasticsearch(
    hosts=[ELASTIC_ENDPOINT], api_key=ELASTIC_API_KEY, request_timeout=3600
)

resp = client.ping()
print(f"Connected to Elastic instance: {resp}")

## Prepare our embedding model workflow

Next we ensure our embedding models are available in Elasticsearch. We will use Elastic's provided `e5_multilingual_small` and `elser_V2` models to provide dense and sparse vectoring, respectively. Using these models out of the box will ensure they are up-to-date and ready for integration with Elasticsearch.

Other models may be uploaded and deployed using [Eland](https://www.elastic.co/docs/reference/elasticsearch/clients/eland) or integrated using the [inference endpoint API](https://www.elastic.co/docs/api/doc/elasticsearch/operation/operation-inference-put-azureopenai) to connect to third-party models.

In [ ]:
# Declare models and endpoint names predeployed by Elastic
elser_model = ".elser_model_2_linux-x86_64"
elser_endpoint = ".elser-2-elasticsearch"

e5_model = ".multilingual-e5-small_linux-x86_64"
e5_endpoint = ".multilingual-e5-small-elasticsearch"

# Define (model, endpoint) tuples to check
model_endpoint_pairs = [(elser_model, elser_endpoint), (e5_model, e5_endpoint)]

# Fetch all loaded models and endpoints once
models = client.ml.get_trained_models()
model_ids = {model["model_id"]: model for model in models["trained_model_configs"]}
endpoints = client.inference.get()
endpoint_ids = {
    endpoint["inference_id"]: endpoint for endpoint in endpoints["endpoints"]
}

# Check each (model, endpoint) pair
for model_id, endpoint_id in model_endpoint_pairs:
    print(f"Checking Model: {model_id}")
    model = model_ids.get(model_id)
    if model:
        print(f"  Model ID: {model['model_id']}")
        print(f"  Description: {model.get('description', 'No description')}")
        print(f"  Version: {model.get('version', 'N/A')}")
    else:
        print("  Model not found or not loaded.")
    print(f"Checking Endpoint: {endpoint_id}")
    endpoint = endpoint_ids.get(endpoint_id)
    if endpoint:
        print(f"  Inference Endpoint ID: {endpoint['inference_id']}")
        print(f"  Task Type: {endpoint['task_type']}")
    else:
        print("  Endpoint not found or not ready.")
    print("------")

### Create an inference pipeline
This function will create an ingest pipeline with inference processors to use `ELSER` (sparse_vector) and `e5_multilingual_small` (dense_vector) to infer against data that will be ingested in the pipeline. This allows us to automatically generate embeddings for the product descriptions when they are indexed into Elasticsearch.

In [ ]:
index_pipeline = "ecommerce-pipeline"
resp = client.ingest.put_pipeline(
    id=index_pipeline,
    processors=[
        {
            "inference": {
                "model_id": elser_endpoint,  # inference endpoint ID
                "input_output": [
                    {
                        "input_field": "description",  # source field
                        "output_field": "elser_description_vector",  # destination vector field
                    }
                ],
            }
        },
        {
            "inference": {
                "model_id": e5_endpoint,  # inference endpoint ID
                "input_output": [
                    {
                        "input_field": "description",  # source field
                        "output_field": "e5_description_vector",  # destination vector field
                    }
                ],
                "inference_config": {"text_embedding": {}},
            }
        },
    ],
)

print(f"ecommerce-pipeline created: {resp['acknowledged']}")

## Index documents
The `ecommerce-search` index we are creating will include fields to support dense and sparse vector storage and search. 

We define the `e5_description_vector` and the `elser_description_vector` fields to store the inference pipeline results. 

The field type in `e5_description_vector` is a `dense_vector`. The `.e5_multilingual_small` model has an embedding size of 384, so the dimension of the vector (dims) is set to 384. 

We also add an `elser_description_vector` field type to support the `sparse_vector` output from our `.elser_model_2_linux-x86_64` model. No further configuration is needed for this field for our use case.

In [ ]:
# define the index name and mapping
commerce_index = "ecommerce-search"
mappings = {
    "properties": {
        "product": {
            "type": "text",
        },
        "description": {
            "type": "text",
        },
        "category": {
            "type": "text",
        },
        "elser_description_vector": {"type": "sparse_vector"},
        "e5_description_vector": {
            "type": "dense_vector",
            "dims": 384,
            "index": "true",
            "similarity": "cosine",
        },
        "e5_semantic_description_vector": {
            "type": "semantic_text",
            "inference_id": e5_endpoint,
        },
        "elser_semantic_description_vector": {"type": "semantic_text"},
    }
}


if client.indices.exists(index=commerce_index):
    client.indices.delete(index=commerce_index)
resp = client.indices.create(
    index=commerce_index,
    mappings=mappings,
)

print(f"Index {commerce_index} created: {resp['acknowledged']}")

### Attach Pipeline to Index
Lets connect our pipeline to the index. This updates the settings of our index to use the pipeline we previously defined as the default.


In [ ]:
resp = client.indices.put_settings(
    index=commerce_index,
    body={"default_pipeline": index_pipeline},
)
print(f"Pipeline set for {commerce_index}: {resp['acknowledged']}")

### Load documents

We load the contents of`products-ecommerce.json` into the `ecommerce-search` index. We will use the `bulk` helper function to efficiently index our documents en masse. 

In [ ]:
# Load the dataset
with open("products-ecommerce.json", "r") as f:
    data_json = json.load(f)


# helper function to create bulk indexing body
def create_index_body(doc):
    doc["elser_semantic_description_vector"] = doc["description"]
    doc["e5_semantic_description_vector"] = doc["description"]

    return {
        "_index": "ecommerce-search",
        "_source": doc,
    }


# prepare the documents array payload
documents = [create_index_body(doc) for doc in data_json]

# use bulk function to index
try:
    print("Indexing documents...")
    resp = bulk(client, documents)
    print(f"Documents indexed successfully: {resp[0]}")
except Exception as e:
    print(f"Error indexing documents: {e}")

## Text Analysis
The classic way documents are ranked for relevance by Elasticsearch based on a text query uses the Lucene implementation of the [BM25](https://en.wikipedia.org/wiki/Okapi_BM25) model, a **sparse model for lexical search**. This method follows the traditional approach for text search, looking for exact term matches.

To make this search possible, Elasticsearch converts **text field** data into a searchable format by performing text analysis.

**Text analysis** is performed by an [analyzer](https://www.elastic.co/guide/en/elasticsearch/reference/current/analyzer-anatomy.html), a set of rules to govern the process of extracting relevant tokens for searching. An analyzer must have exactly one [tokenizer](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-tokenizers.html). The tokenizer receives a stream of characters and breaks it up into individual tokens (usually individual words.) 



### Standard Analyzer
In the example below we are using the default analyzer, the standard analyzer, which works well for most use cases as it provides English grammar based tokenization. Tokenization enables matching on individual terms, but each token is still matched literally.

In [ ]:
# Define the text to be analyzed
text = "Comfortable furniture for a large balcony"

# Define the analyze request
request_body = {"analyzer": "standard", "text": text}  # Stop Analyzer

# Perform the analyze request
resp = client.indices.analyze(
    analyzer=request_body["analyzer"], text=request_body["text"]
)

# Extract and display the analyzed tokens
standard_tokens = [token["token"] for token in resp["tokens"]]
print("Standard-analyzed Tokens:", standard_tokens)

### Stop Analyzer
If you want to personalize your search experience you can choose a different built-in analyzer. For example, by updating the code to use the stop analyzer it will break the text into tokens at any non-letter character with support for removing stop words.

In [ ]:
# Define the analyze request
request_body = {"analyzer": "stop", "text": text}

# Perform the analyze request
response = client.indices.analyze(
    analyzer=request_body["analyzer"], text=request_body["text"]
)

# Extract and display the analyzed tokens
stop_tokens = [token["token"] for token in response["tokens"]]
print("Stop-analyzed Tokens:", stop_tokens)

### Custom Analyzer
When the built-in analyzers do not fulfill your needs, you can create a [custom analyzer](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-custom-analyzer.html)
], which uses the appropriate combination of zero or more  [character filters](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-charfilters.html), a [tokenizer](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-tokenizers.html) and zero or more [token filters](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-tokenfilters.html).

In the below example that combines a tokenizer and token filters, the text will be lowercased by the [lowercase filter](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-lowercase-tokenfilter.html) before being processed by the [synonyms token filter](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-synonym-tokenfilter.html).

> Note: you cannot pass a custom analyzer definition inline to analyze. Define the analyzer in your index settings, then reference it by name in the analyze call. For this reason we will create a temporary index to store the analyzer.

In [ ]:
index_settings = {
    "settings": {
        "analysis": {
            "analyzer": {
                "my_custom_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "char_filter": ["html_strip"],
                    "filter": ["lowercase", "asciifolding"],
                }
            }
        }
    }
}

custom_text = "Čōmføřțǎble Fůrñíturę Fòr â ľarğe Bałcony"

# Create a temporary index with the custom analyzer
client.indices.create(index="temporary_index", body=index_settings)

# Perform the analyze request
resp = client.indices.analyze(
    index="temporary_index", analyzer="my_custom_analyzer", text=custom_text
)

# Extract and display the analyzed tokens
custom_tokens = [token["token"] for token in resp["tokens"]]
print("Custom Tokens:", custom_tokens)

# Delete the temporary index
client.indices.delete(index="temporary_index")

### Text Analysis Results
In the table below, we can observe that analyzers both included with Elasticsearch and custom made may be included with your search requests to improve the quality of your search results by reducing or refining the content being searched. Attention should be paid to your particular use case and the needs of your users.

In [ ]:
print("Standard Token Analyzer")
print(f"Before: \n{text}")
print(f"After: \n{standard_tokens}")
print("===================")
print("Stop Token Analyzer")
print(f"Before: \n{text}")
print(f"After: \n{stop_tokens}")
print("===================")
print("Custom Token Analyzer")
print(f"Before: \n{custom_text}")
print(f"After: \n{custom_tokens}")

## Search 
The remainder of this notebook will cover the following search types:


- Lexical Search
- Semantic Search
    - ELSER Semantic Search (Sparse Vector)
    - E5 Semantic Search (Dense Vector)
    - ELSER Semantic Search with `semantic_text`
    - E5 Semantic Search with `semantic_text`
- Hybrid Search
    - E5 + Lexical (linear combination)
    - E5 + Lexical (RRF)
    - ELSER + Lexical (linear combination)
    - ELSER + Lexical (RRF)
- ES|QL Search
    - Semantic Search ES|QL
    - ELSER ES|QL
    - E5 ES|QL
    - ELSER ES|QL with `semantic_text`
    - E5 ES|QL with `semantic_text`
    

### Lexical Search
Our first search will be a straightforward BM25 text search within the description field. We are storing all of our results in a results_list for a final comparison at the end of the notebook. A convenience function to display the results is also defined.

In [ ]:
results_list = []


def print_search_results(search_results):
    if not search_results:
        print("No matches found")
    else:
        for hit in search_results:
            score = hit["_score"]
            product = hit["_source"]["product"]
            category = hit["_source"]["category"]
            description = hit["_source"]["description"]
            print(
                f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n"
            )


# Regular BM25 (Lexical) Search
resp = client.search(
    size=2,
    index="ecommerce-search",
    query={
        "match": {
            "description": {
                "query": "Comfortable furniture for a large balcony",
                "analyzer": "stop",
            }
        }
    },
    source_excludes=["*_description_vector"],  # Exclude vector fields from response
)

lexical_search_results = resp["hits"]["hits"]
results_list.append({"lexical_search": lexical_search_results})
print_search_results(lexical_search_results)

### Semantic Search with Dense Vector

In [ ]:
response = client.search(
    index="ecommerce-search",
    size=2,
    knn={
        "field": "e5_description_vector",
        "k": 50,  # Number of nearest neighbors to return as top hits.
        "num_candidates": 500,  # Number of nearest neighbor candidates to consider per shard. Increasing num_candidates tends to improve the accuracy of the final k results.
        "query_vector_builder": {  # Object indicating how to build a query_vector. kNN search enables you to perform semantic search by using a previously deployed text embedding model.
            "text_embedding": {
                "model_id": ".multilingual-e5-small-elasticsearch",  # Text embedding model id
                "model_text": "Comfortable furniture for a large balcony",  # Query
            }
        },
    },
    source_excludes=["*_description_vector"],  # Exclude vector fields from response
)

dense_semantic_search_results = response["hits"]["hits"]
results_list.append({"dense_semantic_search": dense_semantic_search_results})
print_search_results(dense_semantic_search_results)

### Semantic Search with Sparse Vector

In [ ]:
# Elastic Learned Sparse Encoder - ELSER

resp = client.search(
    index="ecommerce-search",
    size=2,
    query={
        "sparse_vector": {
            "field": "elser_description_vector",
            "inference_id": ".elser-2-elasticsearch",
            "query": "Comfortable furniture for a large balcony",
        }
    },
    source_excludes=["*_description_vector"],  # Exclude vector fields from response
)


sparse_semantic_search_results = resp["hits"]["hits"]
results_list.append({"sparse_semantic_search": sparse_semantic_search_results})
print_search_results(sparse_semantic_search_results)

### Semantic Search with `semantic_text` Type (ELSER)

In [ ]:
# Elastic Learned Sparse Encoder - ELSER

resp = client.search(
    index="ecommerce-search",
    size=2,
    query={
        "semantic": {
            "field": "elser_semantic_description_vector",
            "query": "Comfortable furniture for a large balcony",
        }
    },
    source_excludes=["*_description_vector"],  # Exclude vector fields from response
)

elser_semantic_text_search_results = resp["hits"]["hits"]
results_list.append({"elser_semantic_text_search": sparse_semantic_search_results})
print_search_results(elser_semantic_text_search_results)

### Semantic Search with `semantic_text` Type (e5)

In [ ]:
# Elastic Learned Sparse Encoder - ELSER

resp = client.search(
    index="ecommerce-search",
    size=2,
    query={
        "semantic": {
            "field": "e5_semantic_description_vector",
            "query": "Comfortable furniture for a large balcony",
        }
    },
    source_excludes=["*_description_vector"],  # Exclude vector fields from response
)

e5_semantic_text_search_results = resp["hits"]["hits"]
results_list.append({"e5_semantic_text_search": e5_semantic_text_search_results})
print_search_results(e5_semantic_text_search_results)

### Hybrid Search - BM25 + `semantic_text` Type

In [ ]:
# BM25 + semantic_text (RRF)
top_k = 2
resp = client.search(
    index="ecommerce-search",
    retriever={
        "rrf": {
            "retrievers": [
                {
                    "standard": {
                        "query": {
                            "match": {
                                "description": "A dining table and comfortable chairs for a large balcony"
                            }
                        }
                    }
                },
                {
                    "standard": {
                        "query": {
                            "semantic": {
                                "field": "e5_semantic_description_vector",
                                "query": "Comfortable furniture for a large balcony",
                            }
                        }
                    }
                },
            ],
            "rank_window_size": 2,
            "rank_constant": 20,
        }
    },
    source_excludes=["*_description_vector"],  # Exclude vector fields from response
)

dense_rrf_search_results = resp["hits"]["hits"]
results_list.append({"dense_rrf_search": dense_rrf_search_results})
print_search_results(dense_rrf_search_results)

### Hybrid Search - BM25 + Dense Vector linear combination

In [ ]:
# BM25 + KNN (Linear Combination)
query = "A dining table and comfortable chairs for a large balcony"
resp = client.search(
    index="ecommerce-search",
    size=2,
    query={
        "bool": {
            "should": [
                {
                    "match": {
                        "description": {
                            "query": query,
                            "boost": 1,
                        }
                    }
                }
            ]
        }
    },
    knn={
        "field": "e5_description_vector",
        "k": 2,
        "num_candidates": 20,
        "boost": 1,
        "query_vector_builder": {
            "text_embedding": {
                "model_id": ".multilingual-e5-small-elasticsearch",
                "model_text": query,
            }
        },
    },
    source_excludes=["*_description_vector"],  # Exclude vector fields from response
)

dense_linear_search_results = resp["hits"]["hits"]
results_list.append({"dense_linear_search": dense_linear_search_results})
print_search_results(dense_linear_search_results)

### Hybrid Search - BM25 + Dense Vector Reverse Reciprocal Fusion (RRF)

[Reciprocal rank fusion](https://www.elastic.co/docs/reference/elasticsearch/rest-apis/reciprocal-rank-fusion) (RRF) is a method for combining multiple result sets with different relevance indicators into a single result set. RRF requires no tuning, and the different relevance indicators do not have to be related to each other to achieve high-quality results.

In [ ]:
# BM25 + KNN (RRF)
top_k = 2
resp = client.search(
    index="ecommerce-search",
    retriever={
        "rrf": {
            "retrievers": [
                {
                    "standard": {
                        "query": {
                            "match": {
                                "description": "A dining table and comfortable chairs for a large balcony"
                            }
                        }
                    }
                },
                {
                    "knn": {
                        "field": "e5_description_vector",
                        "query_vector_builder": {
                            "text_embedding": {
                                "model_id": e5_endpoint,
                                "model_text": "A dining table and comfortable chairs for a large balcony",
                            }
                        },
                        "k": 2,
                        "num_candidates": 20,
                    }
                },
            ],
            "rank_window_size": 2,
            "rank_constant": 20,
        }
    },
    source_excludes=["*_description_vector"],  # Exclude vector fields from response
)

dense_rrf_search_results = resp["hits"]["hits"]
results_list.append({"dense_rrf_search": dense_rrf_search_results})
print_search_results(dense_rrf_search_results)

### Hybrid Search - BM25 + Sparse Vector linear combination

In [ ]:
# BM25 + Elastic Learned Sparse Encoder (Linear Combination)

resp = client.search(
    index="ecommerce-search",
    size=2,
    query={
        "bool": {
            "should": [
                {
                    "match": {
                        "description": {
                            "query": "A dining table and comfortable chairs for a large balcony",
                            "boost": 1,  # You can adjust the boost value
                        }
                    }
                },
                {
                    "sparse_vector": {
                        "field": "elser_description_vector",
                        "inference_id": elser_endpoint,
                        "query": "A dining table and comfortable chairs for a large balcony",
                    }
                },
            ]
        }
    },
    source_excludes=["*_description_vector"],  # Exclude vector fields from response
)

sparse_linear_search_results = resp["hits"]["hits"]
results_list.append({"sparse_linear_search": sparse_linear_search_results})
print_search_results(sparse_linear_search_results)

### Hybrid Search - BM25 + Sparse Vector Reciprocal Rank Fusion (RRF)

In [ ]:
# BM25 + ELSER (RRF)
top_k = 2
resp = client.search(
    index="ecommerce-search",
    retriever={
        "rrf": {
            "retrievers": [
                {
                    "standard": {
                        "query": {
                            "match": {
                                "description": "A dining table and comfortable chairs for a large balcony"
                            }
                        }
                    }
                },
                {
                    "standard": {
                        "query": {
                            "sparse_vector": {
                                "field": "elser_description_vector",
                                "inference_id": elser_endpoint,
                                "query": "A dining table and comfortable chairs for a large balcony",
                            }
                        }
                    }
                },
            ],
            "rank_window_size": 2,
            "rank_constant": 20,
        }
    },
    source_excludes=["*_description_vector"],  # Exclude vector fields from response
)

print(resp["hits"]["hits"])
sparse_rrf_search_results = resp["hits"]["hits"]
results_list.append({"sparse_rrf_search": sparse_rrf_search_results})
print_search_results(sparse_rrf_search_results)

### ES|QL Search
Elastic offers its own query language called ES|QL. ES|QL is a SQL-like query language that allows you to search and analyze data in Elasticsearch. Further information can be found in the [official documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/esql.html).

#### Lexical Search with ES|QL
This demonstrates the lexical search capabilities of ES|QL using the match function. The function `MATCH` specifically searches for matches in a query string within a specified field. In the example below, we search for documents containing the phrase "Comfortable furniture for a large balcony" in the description field.


In [ ]:
""" Convert search_results from es|ql to a dict with _source
    and subproperties of score, description, category, and product """


def normalize_results(search_results):
    normalized_results = []
    results = search_results.body["values"]
    for result in results:
        new_result = {"_source": {}}
        new_result["_score"] = result[-1]
        new_result["_source"]["product"] = result[-2]
        new_result["_source"]["category"] = result[0]
        new_result["_source"]["description"] = result[1]
        normalized_results.append(new_result)

    return normalized_results


esql_query = """
FROM ecommerce-search METADATA _score
| WHERE match(description, "Comfortable furniture for a large balcony")
| SORT _score DESC
| LIMIT 2
"""

resp = client.esql.query(query=esql_query)
esql_lexical_search_results = normalize_results(resp)
results_list.append({"esql_lexical_search": esql_lexical_search_results})
print_search_results(esql_lexical_search_results)

#### Semantic Search with ES|QL
To perform a semantic search using ES|QL, use the `semantic_text` type for your query. This will run a similarity search based on the semantic meaning of the text, rather than the lexical (word-level) matching of the `text` type. Similar to the ease of performing a search with semantic search using the `semantic_text` type with the Python client, the ES|QL query is simple to write and understand.


In [ ]:
esql_query = """
FROM ecommerce-search METADATA _score
| WHERE elser_semantic_description_vector:"Comfortable furniture for a large balcony"
| SORT _score DESC
| LIMIT 2
"""

resp = client.esql.query(query=esql_query)
esql_semantic_search_results = normalize_results(resp)
results_list.append({"esql_semantic_search": esql_semantic_search_results})
print_search_results(esql_semantic_search_results)

## Compiled Results
Here are the results of the previous searches. We can see that all of the results return approximately the same the products.

In [ ]:
# Flatten results for each search type, preserving insertion order
rows = []
for result in results_list:
    search_type = list(result.keys())[0]
    for doc in result[search_type]:
        row = {
            "search_type": search_type,
            "product": doc["_source"].get("product"),
            "category": doc["_source"].get("category"),
            "description": doc["_source"].get("description"),
            "score": doc.get("_score"),
        }
        rows.append(row)

# Create DataFrame without altering row order
df = pd.DataFrame(rows)

# Get the unique search_types in order of appearance
ordered_search_types = []
for row in rows:
    st = row["search_type"]
    if st not in ordered_search_types:
        ordered_search_types.append(st)

for search_type in ordered_search_types:
    group = df[df["search_type"] == search_type]
    display(Markdown(f"### {search_type.replace('_', ' ').title()}"))
    styled = (
        group.drop(columns="search_type")
        .reset_index(drop=True)
        .style.set_properties(
            subset=["description"],
            **{"white-space": "pre-wrap", "word-break": "break-word"},
        )
        .hide(axis="index")  # For pandas >=1.4.0
    )
    display(styled)

As can be seen in the results, the semantic search query provides more relevant results than the lexical search query. This is due to the semantic search query using the `semantic_text` field, which is based on the dense vector representation of the text, while the lexical search query uses the description field, which is based on the lexical representation of the text. Nuances and context are better captured by the semantic search query, making it more effective for finding relevant results.

# Conclusion

It should be noted that while the semantic search query provides more relevant results, it is also more computationally expensive than the lexical search query. This is because the semantic search query requires the calculation of vector representations, which can be computationally intensive. 

Ultimately, it is recommended to use the semantic_text type when implementing semantic search for a few key reasons:
- Query structure is simple and easy to understand.
- Implementing the semantic_text type requires minimal changes to the index mapping and query.
- Setting up an ingest pipeline and inference endpoint is unnecessary.

Using `spare_vector` and `dense_vector` types are more complex and requires additional setup, but can be useful in certain scenarios where semantic search needs to be customized beyond standard semantic text search. This could be a change in the similarity algorithm, use of different vectorization models, or any necessary preprocessing steps. 

Hybrid search retains the power of both lexical and semantic search, allowing for a more flexible and effective search experience. With hybrid search, you can balance the trade-off between relevance and performance, making it a more practical choice for production environments. This should be considered the default approach for search.